In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from scipy import interpolate
from pymatgen.ext.matproj import MPRester
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pymatgen as mg
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,cross_val_score, KFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import seaborn as sns

#-- MATMINER
from matminer.featurizers.conversions import StrToComposition

#-- SKLEARN
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf

#-- SEABORN
import seaborn as sns

# load data

In [ ]:
data_loop_2_final = pd.read_excel('./data_08VRHE.xlsx',index_col=0)#0r 0.63VRHE
data_loop_2_final.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_train_updat_pd,Y_train_updat_pd,test_size=0.2,random_state=0,shuffle=True)
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

# Scaling features

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_ = StandardScaler()
sca_fit_ = scaler_.fit(X_train_np)
X_train_stan_ = sca_fit_.transform(X_train_np)
X_test_stan_ = sca_fit_.transform(X_test_np)

# Tune hypreparameters

In [ ]:
from xgboost import XGBRegressor
param_grid = {'max_depth':np.arange(4,8),
              #'gamma':[0.0,0.2,0.4],
              'min_child_weight':np.arange(2,5),
              'subsample':[0.6,0.8,1],
              'colsample_bytree':[0.6,0.8],
            'reg_alpha': [1,10],
            'reg_lambda': [1,10],
             }
xgb_test =  XGBRegressor(n_estimators=500,
                         #reg_alpha =0.1,
                         #reg_lambda = 0.1,
                         
                        #boosting_type='gbdt',
                        #objective='regression',
                        #min_samples_split=2,
                        learning_rate=0.01,
                        random_state = 0,
                        n_jobs = 2) 
GS = GridSearchCV(xgb_test,param_grid,cv=kfold)
GS.fit(X_train_stan_,y_train_np)
#end = clock()

print(GS.best_params_)
print(GS.best_score_)

# Final model

In [ ]:
from xgboost import XGBRegressor
xgb_test =  XGBRegressor(n_estimators=500,
                        colsample_bytree=0.8,
                        gamma=0.0,
                        max_depth=6,
                         min_child_weight = 2,
                         reg_alpha = 1,
                         reg_lambda = 1,
                         subsample = 1,
                        learning_rate=0.01,
                        random_state = 0,
                        n_jobs = 2) 
score = cross_val_score(xgb_test,X_train_stan_,y_train_np,cv=kfold).mean()
mse = cross_val_score(xgb_test,X_train_stan_,y_train_np,cv=kfold,scoring = 'neg_mean_squared_error').mean()

print(score) # R2
print(mse)                  
print(np.sqrt(score))       
print(np.sqrt(abs(mse)))


# Result of 10-fold

In [ ]:
predicted_xgb = cross_val_predict(xgb_test,X_train_stan_,y_train_np, cv=kfold)

diff_mlpr = abs(np.array(y_train_np)-np.array(predicted_xgb))
diff_mlpr_pd = pd.DataFrame(diff_mlpr)
diff_mlpr_pd.columns = ['diff2']
y_train_pd = pd.DataFrame(y_train_np)
y_train_pd.columns = ['y_true']
predicted_mlpr_pd = pd.DataFrame(predicted_xgb)
predicted_mlpr_pd.columns = ['y_pre']
#X_stan_pd = pd.DataFrame(X_train_stan)
X_stan_pd = X_train.reset_index(drop = True)
data_compare_mlpr = pd.concat([X_stan_pd,y_train_pd,predicted_mlpr_pd,diff_mlpr_pd],axis = 1)
data_compare_mlpr = data_compare_mlpr.sort_values(by='diff2',ascending=False,axis=0)
data_compare_mlpr.to_excel('./xgb_10fold.xlsx')
data_compare_mlpr.head()

# Performance on the training

In [ ]:
clf_xgb = xgb_test.fit(X_train_stan_,y_train_np)
train_xgb = clf_xgb.predict(X_train_stan_)
#X_test_pd_np_gbr = np.array(X_test_pd[index_])
#clf_mlpr.predict(X_test_stan_)
R2 = r2_score(y_train_np,train_xgb)
mse = mean_squared_error(y_train_np,train_xgb)
print(R2)
print(mse)
print(np.sqrt(score))       
print(np.sqrt(abs(mse)))

In [ ]:
diff_xgb = abs(np.array(y_train_np)-np.array(train_xgb))
diff_xgb_pd = pd.DataFrame(diff_xgb)
diff_xgb_pd.columns = ['diff2']
y_train_pd = pd.DataFrame(y_train_np)
y_train_pd.columns = ['y_true']
predicted_xgb_pd = pd.DataFrame(train_xgb)
predicted_xgb_pd.columns = ['y_pre']
#X_stan_pd = pd.DataFrame(X_train_stan)
X_stan_pd = X_train.reset_index(drop = True)
data_compare_xgb = pd.concat([X_stan_pd,y_train_pd,predicted_xgb_pd,diff_xgb_pd],axis = 1)
data_compare_xgb = data_compare_xgb.sort_values(by='diff2',ascending=False,axis=0)
#data_compare_xgb.to_excel('D:/Work in Tohoku/research/a_research_work/c_deepinglearning_ORR/celue4/xgb_training.xlsx')
data_compare_xgb.head(5)

# Performance on the test

In [ ]:
#X_test_pd_np_gbr = np.array(X_test_pd[index_])
R2 = r2_score(y_test_np,clf_xgb.predict(X_test_stan_))
mse = mean_squared_error(y_test_np,clf_xgb.predict(X_test_stan_))
print(R2)
print(mse)
print(np.sqrt(score))       
print(np.sqrt(abs(mse)))

In [ ]:
test_xgb = clf_xgb.predict(X_test_stan_)

diff_mlpr = abs(np.array(y_test_np)-np.array(test_xgb))
diff_mlpr_pd = pd.DataFrame(diff_mlpr)
diff_mlpr_pd.columns = ['diff']
y_train_pd = pd.DataFrame(y_test_np)
y_train_pd.columns = ['y_true']
predicted_mlpr_pd = pd.DataFrame(test_xgb)
predicted_mlpr_pd.columns = ['y_pre']
X_pd = X_test.reset_index(drop = True)
data_compare_mlpr = pd.concat([X_pd,y_train_pd,predicted_mlpr_pd,diff_mlpr_pd],axis = 1)
data_compare_mlpr = data_compare_mlpr.sort_values(by='diff',ascending=False,axis=0)
#data_compare_mlpr.to_excel('D:/Work in Tohoku/research/a_research_work/c_deepinglearning_ORR/celue4/xgb_test.xlsx')
data_compare_mlpr.head()